## setup

In [90]:
! pip install tensorflow numpy matplotlib pandas scikit-learn opencv-python ipywidgets

import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob
import tensorflow as tf
import keras
from keras import Input, Model
from keras.layers import Conv2D, Concatenate, Flatten, Dense, MaxPooling2D
import keras.optimizers
import sklearn
import sklearn.metrics
from PIL import Image
import time
from IPython.display import clear_output, display
import math
import ipywidgets as widgets
from IPython.display import display, clear_output
import cv2




In [91]:
def load_images_from_folder(folder, image_size=(500, 500), numImgs = False):
    paths = sorted(glob(os.path.join(folder, '*.png')) + glob(os.path.join(folder, '*.jpg')))
    paths = sorted(paths, key=lambda x:int(os.path.basename(x).split('.')[0]))

    if numImgs:
        paths = paths[:numImgs]

    images = []
    for path in paths:
        img = keras.preprocessing.image.load_img(path, target_size=image_size)
        img = keras.preprocessing.image.img_to_array(img).astype(np.float32)
        img = img / 255.0  # Normalize to [0,1]
        images.append(img)
    return np.array(images, dtype=np.float32)


# [[distance, pitch, yaw, vehicle_id_string],...]
def load_transforms(folder, numImgs=False):
    paths = sorted(glob(os.path.join(folder, '*.npy')))
    paths = sorted(paths, key=lambda x: int(os.path.basename(x).split('.')[0]))

    if numImgs:
        paths = paths[:numImgs]


    transforms = []
    for path in paths:
        data = np.load(path) 
        transforms.append(data)
    
    return np.array(transforms)





# Create & Transform Texture

In [92]:
# texture transform helper funcs


# texture n_adv_b

# shift

# scale

#3d rot (see below)

#skew / face angle rotation transforms
# x axis rotation

# yaw transforms
# z(vertical) axis rotation
#rotate this to like yaw - main angle of domFace degrees (0,90,180,270)?
# rotate yaw % 90 then something to do with 45 ?
# 0 = rotate 0
# 75 = rotate -15 degrees
# 90 = rotate 0 degrees
#
# yaw % 90 if <= 45 or 90,  else yaw % 90 - 90


# pitch transforms
# x axis rotation

# texture n_adv_p



#map texture to this

# transform map by scale

#sample texture using wrapping aswell (modulo?)

#return this texture?





In [115]:
# all the stuff to calculate the transformation matrices

def rotation_x(theta):
    """Roll: Rotation about the X-axis"""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[ c, 0, s],
                     [ 0, 1, 0],
                     [-s, 0, c]])

def rotation_y(theta):
    """Pitch: Rotation about the Y-axis"""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[1, 0,  0],
                     [0, c, -s],
                     [0, s,  c]])


def rotation_z(theta):
    """Yaw: Rotation about the Z-axis"""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s, 0],
                     [s,  c, 0],
                     [0,  0, 1]])


def getDomFace(yaw, pitch):
    if pitch > 35:
        face='top'
    elif 45 < yaw <= 135:
        face='passenger'
    elif 135 < yaw <= 225:
        face='front'
    elif 225 < yaw <= 315:
        face='driver'
    else:
        face='back'
    
    return face

facePrePitches = {
    "back": 5,
    "front": 20, 
}


def yawDiffCalc(domFace, yaw):
    if domFace == "front":
        return yaw - 180
    elif domFace == "back":
        if yaw > 180:
            return yaw - 360
        else:
            return yaw
    elif domFace == "passenger":
        return yaw - 90
    elif domFace == "driver":
        return yaw - 270
    else:
        return yaw


def calc3dRotMat(pitch, yaw):
    domFace = getDomFace(yaw, pitch)
    yawDiff = yawDiffCalc(domFace, yaw)

    
    yaw_rad = np.radians(yaw)
    

    res_mat = np.eye(3)

    if domFace=='top':
        #yaw by yaw
        top_mat = rotation_z(yaw_rad)
        yawDiff = 0

        res_mat = top_mat @ res_mat

        
    
    elif domFace in ["front", "back"]:
        #yaw by 90
        front_back_yaw_mat = rotation_z(np.radians(90))

        #pitch by facePrePitch
        pre_pitch = np.radians(facePrePitches[domFace])
        pre_pitch_mat = rotation_y(pre_pitch)

        res_mat = pre_pitch_mat @ front_back_yaw_mat @ res_mat


    #roll by yawDiff
    yawDiff_rad = np.radians(yawDiff)
    roll_mat = rotation_x(yawDiff_rad)

    #pitch by pitch
    pitch_rad = np.radians(pitch)
    pitch_mat = rotation_y(pitch_rad)

    res_mat = pitch_mat @ roll_mat @ res_mat

    return res_mat



       

In [106]:

def transform_projection(image, pitch, yaw, distance, shift=(0,0)):

    h, w = image.shape[:2]

    shift_x, shift_y = shift

    scale = 15 - distance

    # ---- Build Projection Matrix ----
    f = 500
    cx, cy = w//2, h//2


    # Define corner points in 3D
    corners_3d = np.array([
        [-w/2, -h/2, 0],
        [ w/2, -h/2, 0],
        [ w/2,  h/2, 0],
        [-w/2,  h/2, 0]
    ])

    # Apply rotation
    R = calc3dRotMat(pitch, yaw)
    rotated = (R @ corners_3d.T).T

    # Apply scaling
    rotated *= scale

    # Project back to 2D
    projected = rotated.copy()
    projected[:,0] = f * projected[:,0] / (f + projected[:,2]) + cx + shift_x
    projected[:,1] = f * projected[:,1] / (f + projected[:,2]) + cy + shift_y

    # Warp perspective
    src_pts = np.array([[0,0],[w,0],[w,h],[0,h]], dtype=np.float32)
    dst_pts = projected[:,:2].astype(np.float32)
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)

    output = cv2.warpPerspective(image, M, (500, 500), flags=cv2.INTER_NEAREST)
    return output.astype(np.float32) / 255.0



In [95]:
def to_float32_unit(image):
    image = np.asarray(image, dtype=np.float32)
    if image.max() > 1.0 or image.min() < 0.0:
        image /= 255.0
    return np.clip(image, 0.0, 1.0).astype(np.float32)


def generateTex(textureStyle, textureResolution):
    if textureStyle == 'noise':
        texture = tf.random.uniform(
            (1, textureResolution, textureResolution, 3),
            minval=0.0,
            maxval=1.0,
            dtype=tf.float32
        )

    elif textureStyle in ['mud', 'snow']:
        img = Image.open(f'textures/{textureStyle}.png') \
                   .convert('RGB') \
                   .resize((textureResolution, textureResolution), resample=Image.NEAREST)

        img = to_float32_unit(img)
        texture = tf.convert_to_tensor(img[None, ...], dtype=tf.float32)

    else:
        raise ValueError(f"Unsupported texture style '{textureStyle}'")

    return texture


def transformTex(texture, out_res=500, shift=None):
    transformed_tex = tf.image.resize(
        texture,
        (out_res, out_res),
        method="bilinear"
    )
    transformed_tex = tf.clip_by_value(transformed_tex, 0.0, 1.0)
    return transformed_tex  # Placeholder for actual transformation logic


In [96]:
def predict_and_display(model, ref, tex, intersection_mask, transformed_tex, original_texture):
    #outputs = model.predict([ref, tex])
    #pred = outputs[0].astype(np.float32)

    # Overlay: only keep intersecting pixels in prediction
    #overlay_preds = np.where(intersection_mask, ref, pred)

    def _imshow(ax, image, title, cmap=None):
        ax.set_title(title)
        if cmap:
            ax.imshow(image, cmap=cmap)
        else:
            ax.imshow(np.clip(image, 0.0, 1.0))
        ax.axis("off")

    fig = plt.figure(figsize=(16, 8))

    _imshow(fig.add_subplot(2, 3, 1), ref[0], "Reference")

    _imshow(fig.add_subplot(2, 3, 2), transformed_tex, "Texture")

    _imshow(fig.add_subplot(2, 3, 3), tex, "Texture Mask")


    tex_overlay = np.where(intersection_mask, ref, tex)
    _imshow(fig.add_subplot(2, 3, 4), tex_overlay[0], "Texture + mask overlay")

    _imshow(fig.add_subplot(2, 3, 5),original_texture.astype(np.float32) / 255.0, "Original Texture")

    #_imshow(fig.add_subplot(2, 3, 5), pred, "Prediction no overlay")

    #_imshow(fig.add_subplot(2, 3, 6), overlay_preds[0], "Prediction + mask overlay")

    plt.tight_layout()
    plt.show()



# Playground

In [97]:
dataset_folder = 'sample_dataset'
numImgsToLoad = 100
sample_references = load_images_from_folder(f"{dataset_folder}/reference", numImgs = numImgsToLoad)
sample_masks = load_images_from_folder(f"{dataset_folder}/masks", numImgs = numImgsToLoad)
sample_overlays = load_images_from_folder(f"{dataset_folder}/overlays", numImgs = numImgsToLoad)
sample_transforms = load_transforms(f"{dataset_folder}/transforms", numImgs = numImgsToLoad)


In [116]:

modelChoice = 'k3_100epch_wo_custom_loss_model.h5'
textureStyle = 'noise'
textureResolution = 256


model = keras.models.load_model(f'models/{modelChoice}', compile=False)

#texture = generateTex(
#    textureStyle,
#    textureResolution
#)

texture = np.random.randint(0, 256, (textureResolution, textureResolution, 3), dtype=np.uint8)

def update_display(sampleNo):
    clear_output(wait=True)
    
    # 1. Grab data for this specific sample
    ref_img = sample_references[sampleNo]
    mask_img = sample_masks[sampleNo]
    overlay_img = sample_overlays[sampleNo]
    transforms = sample_transforms[sampleNo]

    distance = int(transforms[0])
    pitch = int(transforms[1])
    yaw = (int(transforms[2])+90) % 360 # so that yaw 0 is front of car


    # transforms = [[distance, pitch, yaw, vehicle_id_string],...]
    
    transformed_tex = transform_projection(texture,pitch, yaw, distance)

    # Apply texture to masked regions
    mask = tf.cast(
        tf.reduce_any(mask_img > 0.01, axis=-1, keepdims=True),
        tf.float32
    )
    texture_mask = transformed_tex * mask

    refInput = np.expand_dims(ref_img, axis=0).astype(np.float32)  # shape: (1, 500, 500, 3)
    texInput = texture_mask.numpy().astype(np.float32)
    feature_mask = np.expand_dims(overlay_img, axis=0).astype(np.float32)  # shape: (1, 500, 500, 3)

    predict_and_display(model, refInput, texInput, feature_mask, transformed_tex, texture)
    print("[distance, pitch, yaw, vehicle_id_string]" + str(transforms))


slider = widgets.IntSlider(
    value=30,
    min=0, 
    max=len(sample_references) - 1, 
    step=1, 
    description='Sample:',
    continuous_update=False  # Only updates when you release the mouse (prevents lag)
)


out = widgets.interactive_output(update_display, {'sampleNo': slider})
display(slider, out)



IntSlider(value=30, continuous_update=False, description='Sample:', max=99)

Output()